In [4]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np
import os
from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from pathlib import Path
from PIL import Image
import torchvision.transforms.functional as FI


In [5]:
!pip install opendatasets --q

In [6]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/nodoubttome/skin-cancer9-classesisic")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: shubhamjangid2604
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/nodoubttome/skin-cancer9-classesisic


100%|██████████| 786M/786M [00:06<00:00, 133MB/s]


In [7]:
images=[]
labels=[]
for label in os.listdir(r"/content/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Train"):
  for image in os.listdir(rf"/content/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Train/{label}"):
       images.append(rf"/content/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Train/{label}/{image}")
       labels.append(label)

test_images=[]
test_labels=[]
for label in os.listdir("/content/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Test"):
    for image in os.listdir(f"/content/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Test/{label}"):
        test_images.append(f"/content/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Test/{label}/{image}")
        test_labels.append(label)


In [8]:
classes=sorted(set(labels))
class_to_idx={c:i for i,c in enumerate(classes)}
idx_to_class={i:c for i,c in enumerate(classes)}
print(class_to_idx)
print(idx_to_class)

{'actinic keratosis': 0, 'basal cell carcinoma': 1, 'dermatofibroma': 2, 'melanoma': 3, 'nevus': 4, 'pigmented benign keratosis': 5, 'seborrheic keratosis': 6, 'squamous cell carcinoma': 7, 'vascular lesion': 8}
{0: 'actinic keratosis', 1: 'basal cell carcinoma', 2: 'dermatofibroma', 3: 'melanoma', 4: 'nevus', 5: 'pigmented benign keratosis', 6: 'seborrheic keratosis', 7: 'squamous cell carcinoma', 8: 'vascular lesion'}


In [9]:
path=[Path(image) for image in images]
y=labels
train_images,val_images,train_labels,val_labels=train_test_split(path,y,test_size=.2,stratify=y,random_state=42)
print(len(train_images),len(train_labels),len(val_images),len(val_labels))

1791 1791 448 448


In [10]:
def get_min_hw(images):
    min_w,min_h=None,None
    for img in images:
        image=Image.open(img)

        w,h=image.size

        if min_w is None :
            min_w=w
            min_h=h

        else:
            min_w=min(w,min_w)
            min_h=min(h,min_h)
    return min_w,min_h

min_w,min_h=get_min_hw(images)
print(min_w,min_h)

576 450


In [11]:
def pad_to_size(image, target_w, target_h):
    w, h = image.size
    pad_left = (target_w - w) // 2
    pad_right = target_w - w - pad_left
    pad_top = (target_h - h) // 2
    pad_bottom = target_h - h - pad_top
    return FI.pad(image, (pad_left, pad_top, pad_right, pad_bottom), fill=0)

In [48]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]
def get_transform():
  return transforms.Compose([
       transforms.Resize((224,224)),  # shrink so no side is larger
       # transforms.Lambda(lambda img: pad_to_size(img, min_w, min_h)),  # pad to final size
        transforms.ToTensor(),
        transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)

  ])


def get_transform2(target_w,target_h):
   return transforms.Compose([
        transforms.Resize((target_h,target_w)),  # shrink so no side is larger
       transforms.Lambda(lambda img: pad_to_size(img, min_w, min_h)),  # pad to final size
        transforms.ToTensor(),
        transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
   ])

transform=get_transform()
transform2=get_transform2(min_w,min_h)
transform2


Compose(
    Resize(size=(450, 576), interpolation=bilinear, max_size=None, antialias=True)
    Lambda()
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [12]:
class ImageChunk(Dataset):
  def __init__(self,images,labels,transform,class_to_idx):
      super().__init__()
      self.images=images
      self.labels=labels
      self.transform=transform
      self.class_to_idx=class_to_idx
  def __len__(self):
    return len(self.images)

  def __getitem__(self,idx):
     image=self.images[idx]
     label=self.labels[idx]
     image=Image.open(image)
     y=class_to_idx[label]

     image=self.transform(image)
     y=torch.tensor(y,dtype=torch.long)

     return image,y




In [18]:
train_dataset=ImageChunk(train_images,train_labels,transform,class_to_idx)
val_dataset=ImageChunk(val_images,val_labels,transform,class_to_idx)
test_dataset=ImageChunk(test_images,test_labels,transform,class_to_idx)
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader=DataLoader(val_dataset,batch_size=64,shuffle=False)
test_loader=DataLoader(test_dataset,batch_size=64,shuffle=True)

In [49]:
train_dataset2=ImageChunk(train_images,train_labels,transform2,class_to_idx)
val_dataset2=ImageChunk(val_images,val_labels,transform2,class_to_idx)
test_dataset2=ImageChunk(test_images,test_labels,transform2,class_to_idx)
train_loader2=DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader2=DataLoader(val_dataset,batch_size=64,shuffle=False)
test_loader2=DataLoader(test_dataset,batch_size=64,shuffle=True)

In [19]:
from torchvision import models
class ClassifierHead(nn.Module):
    def __init__(self,in_features,num_classes=9,dropout=0.2):
        super().__init__()

        self.fc_1=nn.Linear(in_features,1024)
        self.drop = nn.Dropout(dropout)
        self.fc_2=nn.Linear(1024,256)
        self.fc_3=nn.Linear(256,num_classes)
        self.relu=nn.ReLU()

        nn.init.xavier_uniform_(self.fc_1.weight)
        nn.init.xavier_uniform_(self.fc_2.weight)
        nn.init.xavier_uniform_(self.fc_3.weight)
        nn.init.zeros_(self.fc_1.bias)
        nn.init.zeros_(self.fc_2.bias)
        nn.init.zeros_(self.fc_3.bias)

    def forward(self,x):
        x = self.fc_1(x)

        x = self.relu(x, inplace=True)
        x = self.drop(x)

        x = F.relu(self.fc_2(x), inplace=True)
        return self.fc_3(x)


model=models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
for param in model.features.parameters():
    param.requires_grad=False

num_features=model.classifier.in_features

model.classfier=ClassifierHead(num_features,num_classes=9,dropout=0.2)

def count_trainable_params(model):
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    return trainable, total

print(count_trainable_params(model))


(2339313, 9293169)


In [42]:
model2=models.densenet121(weights=models.DenseNet121_Weights.DEFAULT)
for param in model2.features.parameters():
    param.requires_grad=False

num_features=model2.classifier.in_features

model2.classfier=ClassifierHead(num_features,num_classes=9,dropout=0.2)


In [60]:
loss_fn=nn.CrossEntropyLoss(reduction="mean")


from torch.optim.lr_scheduler import SequentialLR, LinearLR, CosineAnnealingLR

# Suppose you already have your model and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
optimizer2=torch.optim.AdamW(model2.parameters(),3e-4)

# Step 1: Warmup (linearly increase LR)
warmup_epochs = 2
warmup_scheduler = LinearLR(
    optimizer, start_factor=1e-3, end_factor=1.0, total_iters=warmup_epochs
)

# Step 2: Cosine decay
cosine_scheduler = CosineAnnealingLR(optimizer, T_max=5 - warmup_epochs)

# Combine
scheduler = SequentialLR(
    optimizer,
    schedulers=[warmup_scheduler, cosine_scheduler],
    milestones=[warmup_epochs]
)

In [32]:


def train_model(model,train_loader,val_loader,device,num_epochs,optimizer,loss_fn,grad_clip=1.0,use_amp=True):

    model=model.to(device)
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
    train_vary_losses=[]
    val_vary_losses=[]
    train_epoch_losses=[]
    val_epoch_losses=[]

    for epoch in range(num_epochs):

        print(f"epoch start :{epoch}/{num_epochs}")
        model.train()
        epoch_train_loss=0
        for train_step,batch in enumerate(train_loader):
            optimizer.zero_grad(set_to_none=True)
            x,y=batch
            x,y=x.to(device),y.to(device)
            with torch.cuda.amp.autocast(enabled=use_amp):
                logits = model(x)
                loss = loss_fn(logits, y)

            scaler.scale(loss).backward()
            if grad_clip is not None:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            scaler.step(optimizer)
            scaler.update()
            train_vary_losses.append(loss.item())
            epoch_train_loss +=loss.item()
            if(train_step%10==0):
               print(f"train_loss at step {train_step}: {loss.item()}")

        epoch_avg_loss=epoch_train_loss/len(train_loader)
        print(f"epoch {epoch} train_loss : {epoch_avg_loss}")
        train_epoch_losses.append(epoch_avg_loss)

        model.eval()
        with torch.no_grad():
            val_epoch_loss=0
            for val_step, batch in enumerate(val_loader):
                x,y=batch
                x,y=x.to(device),y.to(device)
                logits=model(x)
                loss=loss_fn(logits,y)
                train_vary_losses.append(loss.item())
                if(val_step%5==0):
                    print(f"val loss at step {val_step}: is {loss.item()}")
                val_epoch_loss += loss.item()
            avg_val_loss=val_epoch_loss/len(val_loader)
            val_epoch_losses.append(avg_val_loss)
            print(f"epoch {epoch} val_loss : {avg_val_loss}")
        scheduler.step()
    return train_vary_losses,train_epoch_losses,val_vary_losses,val_epoch_losses

def predict(model,x,idx_to_class,device):
        model.eval()
        with torch.no_grad():

            model=model.to(device)
            x=x.to(device)
            logits=model(x)
            probs=F.softmax(logits,dim=-1)
            idx=torch.argmax(probs,dim=-1)
            idx=idx.tolist()
            result=idx_to_class[idx]
            return result

In [25]:
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [52]:

train_vary_losses,train_epoch_losses,val_vary_losses,val_epoch_losses=train_model(model,train_loader,val_loader,device,5,optimizer,loss_fn,grad_clip=1.0,use_amp=True)

/tmp/ipython-input-1173426260.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)


epoch start :0/5


/tmp/ipython-input-1173426260.py:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):


train_loss at step 0: 1.01568603515625
train_loss at step 10: 0.8876218795776367
train_loss at step 20: 0.9908647537231445
train_loss at step 30: 1.066995620727539
train_loss at step 40: 0.8590338230133057
train_loss at step 50: 1.2210960388183594
epoch 0 train_loss : 0.8978293463587761
val loss at step 0: is 1.1891008615493774
val loss at step 5: is 1.2326570749282837
epoch 0 val_loss : 1.1349454862730843
epoch start :1/5
train_loss at step 0: 0.9871759414672852
train_loss at step 10: 1.023305892944336
train_loss at step 20: 0.7168025970458984
train_loss at step 30: 1.01524019241333
train_loss at step 40: 0.9261641502380371
train_loss at step 50: 0.7033615112304688
epoch 1 train_loss : 0.9169235910688128
val loss at step 0: is 1.1570675373077393
val loss at step 5: is 1.3337581157684326
epoch 1 val_loss : 1.1329245822770255
epoch start :2/5


/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:209: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


train_loss at step 0: 1.2115094661712646
train_loss at step 10: 0.7662982940673828
train_loss at step 20: 0.8190209865570068
train_loss at step 30: 1.072998046875
train_loss at step 40: 0.9972677230834961
train_loss at step 50: 0.9007716178894043
epoch 2 train_loss : 0.9321741142443248
val loss at step 0: is 1.2633501291275024
val loss at step 5: is 1.2182338237762451
epoch 2 val_loss : 1.2050240891320365
epoch start :3/5
train_loss at step 0: 0.9697843194007874
train_loss at step 10: 1.0523834228515625
train_loss at step 20: 0.6765413284301758
train_loss at step 30: 0.896552324295044
train_loss at step 40: 0.5251216888427734
train_loss at step 50: 1.1329154968261719
epoch 3 train_loss : 0.8730191992861884
val loss at step 0: is 1.1657674312591553
val loss at step 5: is 1.2837034463882446
epoch 3 val_loss : 1.1068525654929025
epoch start :4/5
train_loss at step 0: 1.0153261423110962
train_loss at step 10: 0.7070479393005371
train_loss at step 20: 0.615544319152832
train_loss at step 30

In [66]:

train_vary_losses2,train_epoch_losses2,val_vary_losses2,val_epoch_losses2=train_model(model2,train_loader2,val_loader2,device,5,optimizer2,loss_fn,grad_clip=1.0,use_amp=True)

/tmp/ipython-input-1173426260.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)


epoch start :0/5


/tmp/ipython-input-1173426260.py:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):


train_loss at step 0: 1.788238525390625
train_loss at step 10: 1.4385385513305664
train_loss at step 20: 1.5179023742675781
train_loss at step 30: 1.9665145874023438
train_loss at step 40: 1.3192920684814453
train_loss at step 50: 1.3225107192993164
epoch 0 train_loss : 1.3828110247850418
val loss at step 0: is 1.4809550046920776
val loss at step 5: is 1.593348503112793
epoch 0 val_loss : 1.44596494947161
epoch start :1/5
train_loss at step 0: 1.1840991973876953
train_loss at step 10: 1.4332294464111328
train_loss at step 20: 1.3228874206542969
train_loss at step 30: 1.2806549072265625
train_loss at step 40: 1.325632095336914
train_loss at step 50: 1.3089275360107422
epoch 1 train_loss : 1.3263902472598212
val loss at step 0: is 1.4867067337036133
val loss at step 5: is 1.569192886352539
epoch 1 val_loss : 1.4158805438450404
epoch start :2/5
train_loss at step 0: 1.3145933151245117
train_loss at step 10: 1.1413593292236328
train_loss at step 20: 1.1997756958007812
train_loss at step 30

In [67]:
def evaluate_accuracy(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            preds = torch.argmax(logits, dim=1)   # predicted class
            correct += (preds == y).sum().item()  # count correct
            total += y.size(0)                    # total samples

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy * 100:.2f}%")
    return accuracy

In [56]:
evaluate_accuracy(model,test_loader,device)


Test Accuracy: 47.46%


0.4745762711864407

In [68]:
evaluate_accuracy(model2,test_loader2,device)

Test Accuracy: 39.83%


0.3983050847457627